# Introduction

# Install and Import Necessary Libraries

In [1]:
! pip install -U pylibjpeg pylibjpeg-openjpeg pylibjpeg-libjpeg pydicom python-gdcm
! pip install --upgrade pydicom

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ERROR: Could not find a version that satisfies the requirement pylibjpeg (from versions: none)
ERROR: No matching distribution found for pylibjpeg
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [2]:
try:
    import pylibjpeg
except:
    !pip install /kaggle/input/rsna-2022-whl/{pydicom-2.3.0-py3-none-any.whl,pylibjpeg-1.4.0-py3-none-any.whl,python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl}

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Processing /kaggle/input/rsna-2022-whl/pydicom-2.3.0-py3-none-any.whl
Processing /kaggle/input/rsna-2022-whl/pylibjpeg-1.4.0-py3-none-any.whl
Processing /kaggle/input/rsna-2022-whl/python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Attempting uninstall: pydicom
    Found existing installation: pydicom 2.3.1
    Uninstalling pydicom-2.3.1:
      Successfully uninstalled pydicom-2.3.1


In [3]:
!pip install dicomsdl pytorch_lightning timm --no-index --find-links=../input/rbcd-downloads

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in links: ../input/rbcd-downloads
Processing /kaggle/input/rbcd-downloads/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl


In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os
import cv2
from sklearn.metrics import confusion_matrix
import random
from PIL import Image

import pydicom
pydicom.__version__

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset

import timm

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [5]:
base_model = timm.create_model("efficientnetv2_s", num_classes = 1280, pretrained = False, in_chans = 1)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.base_model = base_model
        self.fc1 = nn.Linear(1280, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)


    def forward(self, x):
        x = self.base_model(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [7]:
#model = Net() # On CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Net().to(device) # On GPU
print(model)

Net(
  (base_model): EfficientNet(
    (conv_stem): Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (drop_path): Identity()
        )
        (1): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
 

In [8]:
model.load_state_dict(torch.load('/kaggle/input/rsna-efficientnet-v2-1/EfficientNet V2 and RSNA Screening Mammography 1.pt'))

<All keys matched successfully>

# Prediction of Test Data

In [9]:
RSNA_test_path = '/kaggle/input/rsna-breast-cancer-detection/test_images'

In [10]:
df_test = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
df_test.head()

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id
0,2,10008,736471439,L,MLO,81,0,21,10008_L
1,2,10008,1591370361,L,CC,81,0,21,10008_L
2,2,10008,68070693,R,MLO,81,0,21,10008_R
3,2,10008,361203119,R,CC,81,0,21,10008_R


In [11]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

In [12]:
class RSNA_Testset(Dataset):
    def __init__(self, img_data, img_path, transform = None):
        self.img_path = img_path
        self.transform = transform
        self.img_data = img_data
        
    def __len__(self):
        return len(self.img_data)
    
    def __getitem__(self, index):
        img_name = os.path.join(RSNA_test_path + '/' + str(self.img_data.loc[index, 'patient_id']) + '/' + str(self.img_data.loc[index, 'image_id']) + '.dcm')
        ds = pydicom.dcmread(img_name, force = True)
        image = ds.pixel_array.astype(np.float32)
        image = np.resize(image, (512, 512))
        #image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        #label = torch.tensor(self.img_data.loc[index, 'cancer'])
        if self.transform is not None:
            image = self.transform(image)
        return image#, label

In [13]:
testset = RSNA_Testset(df_test, RSNA_test_path, transform)

In [14]:
model = model.to(device) # On GPU
model.eval()
cancer = []
with torch.no_grad():
    for i in range(len(testset)):
        out = testset[i].unsqueeze_(0)
        out = out.to(device)
        out = model(out)
        out = torch.sigmoid(out)
        cancer.append(out.item())
cancer

[0.634617805480957, 0.4802100658416748, 0.5621020793914795, 0.4504207372665405]

In [15]:
'''
# Load the test set.
testset = RSNA_Testset(df_test, RSNA_test_path, transform)

# Create a DataLoader for the test set.
testloader = torch.utils.data.DataLoader(testset, batch_size = 32, shuffle = False)

# Move the model to the GPU (if available).
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Put the model in evaluation mode.
model.eval()

# Make predictions on the test set.
cancer = []
with torch.no_grad():
    for images in testloader:
        # Move the batch of images to the GPU (if available).
        images = images.to(device)

        # Pass the batch of images through the model to get the predicted cancer probabilities.
        out = model(images)

        # Apply a sigmoid function to get a probability between 0 and 1.
        out = torch.sigmoid(out)

        # Add the predicted probabilities to the list
        cancer.extend(out.cpu().numpy())
        
        cancer_probs = np.concatenate(cancer)
        
        cancer = [x[0] for x in cancer]

# Print the predicted probabilities.
print(cancer)
'''

'\n# Load the test set.\ntestset = RSNA_Testset(df_test, RSNA_test_path, transform)\n\n# Create a DataLoader for the test set.\ntestloader = torch.utils.data.DataLoader(testset, batch_size = 32, shuffle = False)\n\n# Move the model to the GPU (if available).\ndevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")\nmodel = model.to(device)\n\n# Put the model in evaluation mode.\nmodel.eval()\n\n# Make predictions on the test set.\ncancer = []\nwith torch.no_grad():\n    for images in testloader:\n        # Move the batch of images to the GPU (if available).\n        images = images.to(device)\n\n        # Pass the batch of images through the model to get the predicted cancer probabilities.\n        out = model(images)\n\n        # Apply a sigmoid function to get a probability between 0 and 1.\n        out = torch.sigmoid(out)\n\n        # Add the predicted probabilities to the list\n        cancer.extend(out.cpu().numpy())\n        \n        cancer_probs = np.concatenat

In [16]:
df_test['cancer'] = cancer
df_test

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id,cancer
0,2,10008,736471439,L,MLO,81,0,21,10008_L,0.634618
1,2,10008,1591370361,L,CC,81,0,21,10008_L,0.480210
2,2,10008,68070693,R,MLO,81,0,21,10008_R,0.562102
3,2,10008,361203119,R,CC,81,0,21,10008_R,0.450421


In [17]:
submission = df_test.loc[:, 'prediction_id':'cancer']
submission

,prediction_id,cancer
0,10008_L,0.634618
1,10008_L,0.480210
2,10008_R,0.562102
3,10008_R,0.450421


In [18]:
submission = submission.groupby('prediction_id').mean().reset_index()
submission

,prediction_id,cancer
0,10008_L,0.557414
1,10008_R,0.506261


In [19]:
submission = pd.DataFrame(data = {'prediction_id': df_test['prediction_id'], 'cancer': cancer})
submission

,prediction_id,cancer
0,10008_L,0.634618
1,10008_L,0.480210
2,10008_R,0.562102
3,10008_R,0.450421


In [20]:
submission = submission.groupby('prediction_id').max().reset_index()
submission

,prediction_id,cancer
0,10008_L,0.634618
1,10008_R,0.562102


In [21]:
submission.to_csv('submission.csv', index = False)

## Credits

Please go and visit these below mentioned notebooks and support their works and advices. Without the help of the below mentioned notebooks and advices, it would have been much difficult for me to approach the solution.
1. [how to read .dcm (DICOM) data](https://www.kaggle.com/code/micheldc55/how-to-read-dcm-dicom-data)
2. [PyTorch-Tutorial (The Classification)](https://www.kaggle.com/code/basu369victor/pytorch-tutorial-the-classification)
3. [FFT with resnet50](https://www.kaggle.com/code/kilogrand/fft-with-resnet50/notebook#import)
4. [rsna-2022-whl](https://www.kaggle.com/code/vslaykovsky/rsna-2022-whl)
5. [RBCD Downloads](https://www.kaggle.com/code/clemchris/rbcd-downloads)
6. [Thomas Konstantin](https://www.kaggle.com/code/gokifujiya/pytorch-and-rsna-screening-mammography/comments)